# Import All Necessary Modules And Setup Project

If you get any errors when importing these, ensure you run the commands:
```bash
$ python -m pip install -r requirements.txt
```
to install all necessary modules for this project. This command must be run from inside of this project directory.

It is recommended to use virtual environments for this project to ensure there is no conflicting package versions on your system.

Activate the virtual environment (if needed), run the pip install command, and then launch Jupyter Lab inside this project to get this project running.

In [ ]:
# Uncomment the following line to execute the pip install
# %pip install -r requirements.txt

In [ ]:
import pandas as pd
import numpy as np

# Visualization
from matplotlib import pyplot as plt
import seaborn as sns

from measure_incremental_development.compute import calculate_mid, classify_snapshots


## Import Auxiliary Modules and Functions

In [ ]:
from projectConstants import *

`projectConstants` defines various constants (namely, column names) that are used throughout the project.

In [ ]:
from getSubmissionDataframes import *

`getSubmissionDataframes` contains the following functions:

*   `getFileInStudentSubmission`
*   `getStudentSubmission`
*   `filterDownToRunAndEdits`
*   `filterDownToRunAndEditsAndPastes`
*   `getStudentSubmissionRunsAndEdits`
*   `getFileInStudentSubmissionRunsAndEdits`

In [ ]:
from reconstructSubmissions import *

`reconstructSubmissions` has the functions:

*   `reconstructSingleFileDebugger`
*   `reconstructFinalFile`
*   `reconstructFileAtRunEvents`
*   `reconstructProjectAtRunEvents`

In [ ]:
from viewReconstructions import *

`viewReconstructions` has the following functions:

*   `viewFinalReconstructedProject`
*   `viewReconstructedProjectStates`

In [ ]:
from getStudentProjectInfo import *

`getStudentProjectInfo` has the following function:

*   `getStudentProjectList`

In [ ]:
from filterOutBadReconstructions import *

`filterOutBadReconstructions` has the following functions:

*   `getFileReconstructionDF`
    *   Get the raw file reconstructions dataframe
*   `getProjectReconstructionDF`
    *   Same as above, but if any file in a project fails at reconstructing, the whole project is marked as a failed reconstruction
*   `getOnlyBadFileReconstructionsDF`
    *   Get a DF like `getFileReconstructionDF` returns, containing *ONLY* the bad file reconstructions
*   `getOnlyBadProjectReconstructions`
    *   Get a DF like `getProjectReconstructionDF` returns, containing *ONLY* the bad project reconstructions
*   `mergeKeystrokesWithFileReconstructions`
    *   Merge a keystroke dataframe with the *file* reconstruction df on `SubjectID, AssignmentID, CodeStateSection`
*   `mergeKeystrokesWithProjectReconstructions`
    *   Merge a keystroke dataframe with the *project* reconstruction df on `SubjectID, AssignmentID`
*   `getKeystrokesDFWithoutBadFileReconstructions`
    *   Filter down the keystroke dataframe to remove information related to *files* that reconstruct incorrectly
*   `getKeystrokesDFWithoutBadProjectReconstructions`
    *   Filter down the keystroke dataframe to remove information related to *projects* that reconstruct incorrectly

Unless granularity of the keystroke data is desired, the `getKeystrokesDFWithoutBadProjectReconstructions` will probably be the only needed function.

NOTE: The reconstruction data used for these functions was generated by the `checkSubmissions.sh` script and `determineReconstructionFailures.ipynb` notebook.

In [ ]:
from midScoreFunctions import *

`midScoreFunctions` has the following functions:

*   `remove_empty_at_start`
*   `get_scores`
*   `get_mid_score_row`
*   `get_mid_score_all`

In [ ]:
from timeBetweenRuns import *

`timeBetweenRuns` has the following functions:

*   `getTimestampRow`
*   `getFilteredRunEvents`
*   `getTimeBetweenRuns`
*   `getTimeBetweenRunsDf`

In [ ]:
# from codingSessionFunctions import *
import codingSessionFunctions

`codingSessionFunctions` has the following functions:

*   `getCodingSessionsForAssignment`
*   `getTimeSpentPerSession`
*   `getTimeSpentPerAssignment`
*   `getCodingSessionsDf`

## Load Datasets

In [ ]:
keystroke_df_unedited = pd.read_csv("data/keystrokes.csv")
student_df_unedited = pd.read_csv("data/students.csv")

#### Copy Datasets For Modification

This preserves the initial datasets, in case we ever need to bring an unedited column/row back into anything

In [ ]:
keystroke_df = keystroke_df_unedited.copy()
student_df = student_df_unedited.copy()

#### Filter Keystroke Data To Only Projects That Have Reconstructed Correctly

In [ ]:
keystroke_df = getKeystrokesDFWithoutBadProjectReconstructions(keystroke_df)

#### Get Information And Keystroke Dataframes For Each `Student,Project` pair

**NOTE:** This may take a few minutes to compute. 

In [ ]:
projects_df, run_events_df, final_data = getStudentProjectList(student_df, keystroke_df)

print(len(projects_df), len(run_events_df))

In [ ]:
# List all students with a submission for the assignment
BREAK
for student, assign, df in final_data:
    if len(df) > 0:
        print(student, assign)

In [ ]:
for student, assign, df in final_data:
    if len(df) > 0:
        # print(len(df))
        print(50*'=')
        print(student, assign)
        viewFinalReconstructedProject(df)
        print(50*'=')


## Coding Session Functions

In [ ]:
from projectConstants import *
import pandas as pd
import numpy as np

from timeBetweenRuns import *

SESSION_COUNT_KEY = 'SessionCount'
FIRST_SESSION_START_KEY = 'FirstSessionStart'
LAST_SESSION_END_KEY = 'LastSessionEnd'
NUMBER_KEYSTROKES_KEY = 'NumberKeystrokes'
AVG_KEYDIFF_TIME_KEY = 'AvgKeyDiffTime'

# use complete keystroke data
# student, assignment, number of coding sessions
def markEventsByCodingSessions(df, student, assignment):
    '''
    For a specific (student, assignment) pair, make a new dataframe that marks all these events with a coding session

    Coding sessions end if there's more than 5 minutes from last event
    '''
    assnSessionDf = df[
        (df.SubjectID == student) & (df.AssignmentID == assignment)
        & (
        # We do not want any X-Attention events or Editor Close Actions
          ~ (
              ( df.EventType == 'X-Attention' )
            | ( ( df.EventType == 'X-Action' ) & (df['X-Metadata'] == 'Close Active Editor')
            )
          )
        )
        ].copy()
    # assnSessionDf = assnSessionDf[SUBJECT_ID_KEY, ASSIGNMENT_ID_KEY, CLIENT_TIMESTAMP_KEY]
    # convert client timestamp to date time
    assnSessionDf[DATE_TIME_KEY] = pd.to_datetime(assnSessionDf.ClientTimestamp, unit='ms') 
    assnSessionDf[SESSION_ID_KEY] = -1
    assnSessionDf[EVENT_TIME_DIFF_KEY] = np.NaN
    # assnSessionDf[EVENT_TIME_DIFF_KEY].astype(float)

    # sort by timestamp
    assnSessionDf.sort_values(by=DATE_TIME_KEY, inplace=True)
    # give session number , if within 5 minutes, keep that session number numbers
    if assnSessionDf.size > 0:
        lastEventTime = assnSessionDf.head(1)[DATE_TIME_KEY].values[0]
    # 5 minutes from last event, not start of the session
    sessionId = 0
    for i, row in assnSessionDf.iterrows():
        diff = row[DATE_TIME_KEY] - lastEventTime
        minutes_diff = diff.total_seconds() / 60
        if minutes_diff > 5:
            sessionId += 1
            # assnSessionDf.at[i, EVENT_TIME_DIFF_KEY] = np.NaN
        else:
            # If this event is still part of the same coding session, we note the diff time
            # If it's not, leave it as NaN
            assnSessionDf.at[i, EVENT_TIME_DIFF_KEY] = diff.total_seconds()
        assnSessionDf.at[i,SESSION_ID_KEY] = sessionId
        lastEventTime = row[DATE_TIME_KEY]

    # Add time diff between events column in this function
        #   First event has NaN
        #   Other events have NNaN

    # Do I reduce the DF here?
    return assnSessionDf


def getIndividualSessionInfo(df):
    '''
    Get information for each coding sesssion for a given (Student,Assignment)

    df should contain keystroke/event data for a given student/assignment with session numbers and time-diffs between sessions marked
        (See markEventsByCodingSessions)
    
    '''
    if df.size == 0:
        return None

    # This dataframe should only be for one (Student,Assignment)

    allSessionIds = df[SESSION_ID_KEY].unique()
    student = df.head(1)[SUBJECT_ID_KEY].values[0]
    assn = df.head(1)[ASSIGNMENT_ID_KEY].values[0]

    studentSessionID = [student] * len(allSessionIds)
    assnSessionID = [assn] * len(allSessionIds)

    sessionIdsColumn = []
    sessionStartTimes = []
    sessionEndTimes = []
    sessionLengths = []
    sessionNumEvents = []
    sessionAvgKeyDiffTime = []

    for sessionId in allSessionIds:
        # Reduce df down to just this session
        sessionDf = df[(df[SESSION_ID_KEY] == sessionId)].copy()
        # sort by dates just in case
        sessionDf.sort_values(by=DATE_TIME_KEY, inplace=True)
        
        startTime = sessionDf.head(1)[CLIENT_TIMESTAMP_KEY].values[0]
        endTime = sessionDf.tail(1)[CLIENT_TIMESTAMP_KEY].values[0]
        # subtract start session time and end session time to get total session time
        lengthTime = endTime - startTime
        
        avgKeyDiffTime = sessionDf[EVENT_TIME_DIFF_KEY].mean()

        # Add all data for this session to a list
        sessionIdsColumn.append(sessionId)
        sessionStartTimes.append(startTime)
        sessionEndTimes.append(endTime)
        sessionLengths.append(lengthTime)
        sessionNumEvents.append(len(sessionDf))
        sessionAvgKeyDiffTime.append(avgKeyDiffTime)

        # codingSessionDf = pd.concat([codingSessionDf, sessionDf], ignore_index=True)
    return pd.DataFrame(
        {
            SUBJECT_ID_KEY: studentSessionID,
            ASSIGNMENT_ID_KEY: assnSessionID,
            SESSION_ID_KEY: sessionIdsColumn,
            SESSION_START_TIME_KEY: sessionStartTimes,
            SESSION_END_TIME_KEY: sessionEndTimes,
            SESSION_TIME_KEY: sessionLengths,
            SESSION_KEYSTROKES_KEY: sessionNumEvents,
            SESSION_AVG_KEYDIFF_TIME_KEY: sessionAvgKeyDiffTime
        }
    )

def getCodingSessionsDf(keystroke_df, final_data):
    '''Get coding session for each student and assignment, where coding session is within 5 minutes'''
    # Make this function take all keystrokes, not just run events
    # Make another function that finds the time between run events
    # runEvents = getFilteredRunEvents(keystroke_df)
    codingSessionDf = pd.DataFrame()
    for student, assignment, _ in final_data:

        # Get a dataframe with keystroke info for this (student,assignment), marked by coding sessions
        codingSessionAssignmentDf = markEventsByCodingSessions(keystroke_df, student, assignment)

        # Compute info for each coding session
        oneCodingSessionInfoDf = getIndividualSessionInfo(codingSessionAssignmentDf)

        if oneCodingSessionInfoDf is None:
            continue

        codingSessionDf = pd.concat([codingSessionDf, oneCodingSessionInfoDf], ignore_index=True)

    return codingSessionDf


def sessionInfoToAssignmentInfo(session_info_df):
    '''
    Convert a dataframe containing individual session info into a dataframe containing 
    info for all sessions on an assignment
    '''

    return session_info_df.groupby(
        by=[SUBJECT_ID_KEY, ASSIGNMENT_ID_KEY]
        ).agg({
            SESSION_ID_KEY: 'count',
            SESSION_TIME_KEY: 'sum',
            SESSION_START_TIME_KEY: 'first',
            SESSION_END_TIME_KEY: 'last',
            SESSION_KEYSTROKES_KEY: 'sum',
            SESSION_AVG_KEYDIFF_TIME_KEY: 'mean',
        }).rename(
            columns={
                SESSION_ID_KEY: SESSION_COUNT_KEY,
                SESSION_START_TIME_KEY: FIRST_SESSION_START_KEY,
                SESSION_END_TIME_KEY: LAST_SESSION_END_KEY,
                SESSION_TIME_KEY: TOTAL_ASSIGNMENT_TIME_KEY,
                SESSION_KEYSTROKES_KEY: NUMBER_KEYSTROKES_KEY,
                SESSION_AVG_KEYDIFF_TIME_KEY: AVG_KEYDIFF_TIME_KEY
            }).reset_index()


## Add MID Library

- 0-2 Likely Incremental
- 2-2.5 Somewhat Incremental
- 2.5-3 Somewhat Non-Incremental
- 3+ Likely Non-Incremental

#### Calculate MID statistc for student and assignmemt

In [ ]:
print(get_scores('Student10', 'Assign10', student_df))

In [ ]:
mid_df = get_mid_score_all(final_data, student_df)

In [ ]:
mid_df.to_csv('./data/mid_scores.csv', index=False)

## Code to get the time between runs

In [ ]:
runEvents = getFilteredRunEvents(keystroke_df)

In [ ]:
timeBetweenRunsDf = getTimeBetweenRunsDf(keystroke_df, final_data)

In [ ]:
display(timeBetweenRunsDf)

In [ ]:
timeBetweenRunsDf.to_csv('./data/timeBetweenRuns.csv', index=False)

## Get Coding Sessions
#### Defined as keypresses within 5 minutes of eachother

NOTE: The above may need to be double checked to ensure all keypresses are captured

In [ ]:
codingSessions = getCodingSessionsDf(keystroke_df, final_data)

In [ ]:
codingSessions.to_csv('./data/codingSessions.csv', index=False)

In [ ]:
assignmentKeystrokeInfo = sessionInfoToAssignmentInfo(codingSessions)

In [ ]:
assignmentKeystrokeInfo.to_csv('./data/assignmentKeystrokeInfo.csv', index=False)